In [ ]:
#Crear una tabla en Lakehouse usando PySpark
# filepath: raw_nb_gts_validarcargapredictores/creacion_tabla_lakehouse.ipynb

# 1. Importar librerías
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [ ]:
# 2. Crear sesión de Spark
spark = SparkSession.builder.getOrCreate()

In [ ]:
# 3. Definir el esquema de la tabla
# Nota: id e id2 actuarán como clave primaria compuesta (no soportado nativamente en Delta Lake)
schema = StructType([
    StructField("pk_id", StringType(), True),
    StructField("id", IntegerType(), True),
    StructField("id2", IntegerType(), True),
    StructField("nombre", StringType(), True),
    StructField("valor", IntegerType(), True)
])


In [ ]:
# 4. Crear un DataFrame de ejemplo
df= spark.createDataFrame([], schema)

In [ ]:
# 5. Escribir el DataFrame como tabla en el Lakehouse (formato Delta)
lakehouse_path = "Tables/mi_tabla_lakehouse"
df.write.format("delta").mode("append").option("mergeSchema", "true").save(lakehouse_path)

In [ ]:
# 6. Registrar la tabla en el catálogo de Spark (opcional)
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS mi_tabla_lakehouse
    (
        pk_id STRING,
        -- Clave primaria compuesta (no soportada nativamente, solo informativa)
        id INT,
        id2 INT,
        nombre STRING,
        valor INT
    )
    USING DELTA
    LOCATION '{lakehouse_path}'
    COMMENT 'PRIMARY KEY (id, id2) - No soportado nativamente, solo informativo'
""")

In [ ]:
# 7. Consultar la tabla creada
df_leida = spark.read.format("delta").load(lakehouse_path)
df_leida.show()